In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
#import os
#os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!kaggle datasets download -d vikashrajluhaniwal/fashion-images

In [ ]:
import zipfile

file_path = 'archive.zip'

with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('downloads/archive')

In [ ]:
#!pip install tensorflow
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

**Import Model**

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable=False

model=tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])
print(model.summary())

**Feature Extraction**

In [ ]:
import os

def extract_features(img_path,model):
  img=image.load_img(img_path,target_size=(224,224))
  img_arr=image.img_to_array(img)
  expanded_arr=np.expand_dims(img_arr,axis=0)
  preprocessing_img=preprocess_input(expanded_arr)
  result=model.predict(preprocessing_img).flatten()
  normalized_result=result/norm(norm(result))

  return normalized_result


In [ ]:
import os

dir_path='data/Apparel/Boys/Images/images_with_product_ids'
img_list=os.listdir(dir_path)
print(img_list)

['4203.jpg', '38491.jpg', '24906.jpg', '24912.jpg', '36325.jpg', '39002.jpg', '36319.jpg', '39758.jpg', '34056.jpg', '34042.jpg', '31112.jpg', '40933.jpg', '31106.jpg', '40927.jpg', '5486.jpg', '31099.jpg', '3815.jpg', '36721.jpg', '38283.jpg', '39837.jpg', '41754.jpg', '41967.jpg', '37173.jpg', '37167.jpg', '37601.jpg', '37600.jpg', '37172.jpg', '35995.jpg', '41966.jpg', '38282.jpg', '24873.jpg', '37199.jpg', '36720.jpg', '3814.jpg', '31098.jpg', '40926.jpg', '31107.jpg', '31113.jpg', '34043.jpg', '34057.jpg', '46833.jpg', '39759.jpg', '33260.jpg', '36318.jpg', '39003.jpg', '36324.jpg', '24913.jpg', '38490.jpg', '24907.jpg', '4202.jpg', '21843.jpg', '4200.jpg', '8348.jpg', '24911.jpg', '38492.jpg', '38479.jpg', '39001.jpg', '36326.jpg', '46831.jpg', '33262.jpg', '34096.jpg', '35439.jpg', '34041.jpg', '34055.jpg', '22637.jpg', '31105.jpg', '40924.jpg', '31111.jpg', '43587.jpg', '10054.jpg', '40139.jpg', '36722.jpg', '3816.jpg', '4189.jpg', '39834.jpg', '41970.jpg', '35997.jpg', '33302.

In [ ]:
print(len(img_list))

759


In [ ]:
from tqdm import tqdm

feature_list = []

for img_file in tqdm(img_list, desc='Extracting features'):
    features = extract_features(os.path.join(dir_path, img_file), model)
    feature_list.append(features)

pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(img_list,open('filenames.pkl','wb'))

In [ ]:
import pickle
from tensorflow.keras.preprocessing import image

feature_list=pickle.load(open('embeddings.pkl','rb'))
filenames=pickle.load(open('filenames.pkl','rb'))
#print(np.array(feature_list).shape)
img=image.load_img('/content/Test_sample/18170.jpg',target_size=(224,224))
img_arr=image.img_to_array(img)
expanded_arr=np.expand_dims(img_arr,axis=0)
preprocessing_img=preprocess_input(expanded_arr)
result=model.predict(preprocessing_img).flatten()
normalized_result=result/norm(norm(result))

print((normalized_result).shape)

In [ ]:
from sklearn.neighbors import NearestNeighbors

neighbors=NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
neighbors.fit(feature_list)

distances,indices= neighbors.kneighbors([normalized_result])

print(distances,indices,sep='\n')



In [ ]:
%pip install streamlit

In [ ]:
%%writefile main.py

import streamlit as st
import os
from PIL import Image
import pickle
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm



feature_list=pickle.load(open('embeddings.pkl','rb'))
filenames=pickle.load(open('filenames.pkl','rb'))

model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable=False

model=tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

st.write('Fashion Recommender System')
def save_uploaded_file(uploaded_file):
  try:
    #here we need to make uploads directory
    with open(os.path.join('uploads',uploaded_file.name),'wb') as f:
      f.write(uploaded_file.getbuffer())
    return 1
  except:
    return 0

def feature_extraction(img_path,model):
  img=image.load_img(img_path,target_size=(224,224))
  img_arr=image.img_to_array(img)
  expanded_arr=np.expand_dims(img_arr,axis=0)
  preprocessing_img=preprocess_input(expanded_arr)
  result=model.predict(preprocessing_img).flatten()
  normalized_result=result/norm(norm(result))

  return normalized_result

def recommend(features,feature_list):
  neighbors=NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
  neighbors.fit(feature_list)
  distances,indices= neighbors.kneighbors([features])
  return indices

In [ ]:
%npm install localtunnel

In [ ]:
%streamlit run main.py &>/content/logs.txt &

OSError: Background processes not supported.

In [ ]:
%npx localtunnel --port 8501 & curl ipv4.icanhazip.com

In [ ]:
import streamlit as st

uploaded_file = st.file_uploader("Choose an image")
if uploaded_file is not None:
  if save_uploaded_file(uploaded_file):
    # file has been uploaded
    display_image = Image.open(uploaded_file)

    #display the file
    st.image(display_image)

    #feature extract
    feature_extraction(os.path.join("uploads",uploaded_file.name),model)
    st.text(features)

    #recommendation
    indices = recommend(features,feature_list)

    #show

    col1,col2,col3,col4,col5 = st.beta_columns(5)

    with col1:
      st.image(filenames[indices[0][0]])
    with col2:
      st.image(filenames[indices[0][1]])
    with col3:
      st.image(filenames[indices[0][2]])
    with col4:
      st.image(filenames[indices[0][3]])
    with col5:
      st.image(filenames[indices[0][4]])

  else:
    st.header("Some error occured in file upload")

In [ ]:
%streamlit run /Users/karnavshah/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py


      👋 Welcome to Streamlit!

      If you’d like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  